<a href="https://colab.research.google.com/github/audachang/ML_collections/blob/main/face_attr_googlenet_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Paths
dpath = './drive/MyDrive/mycolab/data/'
model_path = './drive/MyDrive/mycolab/model/'
imgdir = os.path.join(dpath, '346faces_sameBG_grey_adjBright')
annote_file = './drive/MyDrive/mycolab/data/346faces_allData.csv'  # or whatever the path to the downloaded data is

# Training parameters
num_classes = 2
learning_rate = 3e-4
batch_size = 32
num_epochs = 10
IMAGE_WIDTH = 480
IMAGE_HEIGHT = 480
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

# Load the dataset and split into training and testing sets
annotations = pd.read_csv(annote_file)
annotations['value'] = annotations['gray_att_upr']
train_df = annotations.sample(frac=0.8, random_state=42)
test_df = annotations.drop(train_df.index)

# Image data generator
def preprocess_image(image):
    image = image / 255.0
    return image

datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    horizontal_flip=True,
    preprocessing_function=preprocess_image,
)

train_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=imgdir,
    x_col='image_grey_adj',
    y_col='value',
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='raw',
)

test_gen = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=imgdir,
    x_col='image_grey_adj',
    y_col='value',
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='raw',
)

def create_inceptionv3_regressor():
    base_model = tf.keras.applications.InceptionV3(input_shape=(*IMAGE_SIZE, 3), include_top=False, pooling='avg')
    x = base_model.output
    predictions = layers.Dense(1)(x)
    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

# Instantiate the inceptionv3 regressor model, criterion, and optimizer
model = create_inceptionv3_regressor()
criterion = losses.MeanSquaredError()
optimizer = optimizers.Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss=criterion)

# Train the model
history = model.fit(
    train_gen,
    epochs=num_epochs,
    steps_per_epoch=len(train_gen),
    validation_data=test_gen,
    validation_steps=len(test_gen),
)

# Save the final model
model.save(os.path.join(model_path, 'tf.inceptionv3_pretrained-Final.h5'))

# Test the model
test_loss = model.evaluate(test_gen, steps=len(test_gen))
print('Test Loss: {:.4f}'.format(test_loss))

# Create a scatter plot of predicted vs target values
test_gen.reset()
predicted_values = model.predict(test_gen, steps=len(test_gen))
target_values = np.concatenate([test_gen[i][1] for i in range(len(test_gen))], axis=0)
plt.scatter(target_values, predicted_values)
plt.xlabel('Target values')
plt.ylabel('Predicted values')
plt.show()


In [ ]:
train_df